**Solución ejercicios base de datos employees en SQL en Python**


por Sibyl González

email: sibylgonzalez@gmail.com

In [1]:
import pandas as pd
import numpy as np
import mysql.connector as conn
from sqlalchemy import create_engine 


In [2]:
# Conexión a la base de datos: usuario y contraseña de workbench.
db_conect=conn.connect(host='localhost', user='*******', passwd='*********', database='employees')

In [ ]:
cursor=db_conect.cursor() # cursor para conectarse a la base de datos

In [ ]:
# Prueba para ver si estamos conectados a la base de datos correcta.
cursor.execute('show tables;')

for x in cursor:
    print(x)

# Ejercicio 1.

    Obtener la cantidad de empleados por género de cada departamento.

In [ ]:
gender_employees_per_department = pd.read_sql('''
SELECT DISTINCT
    d.dept_name AS departments,
    COUNT(CASE WHEN e.gender = 'M' THEN 1 END) AS male,
    COUNT(CASE WHEN e.gender = 'F' THEN 1 END) AS female
FROM
    departments d
JOIN
    dept_emp de ON d.dept_no = de.dept_no
JOIN
    employees e ON de.emp_no = e.emp_no
GROUP BY
    d.dept_name

    ;
    ''', con=db_conect)
gender_employees_per_department

# Ejercicio 2

    Obtén los nombres de los managers activos y el de su predecesor por cada
    departamento.

In [ ]:
deparment_managers = pd.read_sql('''
Select Deparment, Manager, Previous_Manager from
(SELECT d.dept_name AS department,CONCAT(m.first_name, ' ', m.last_name) AS current_manager,
CONCAT(p.first_name, ' ', p.last_name) AS predecessor_manager
FROM
    departments d
JOIN
    dept_manager dm ON d.dept_no = dm.dept_no
JOIN
    employees m ON dm.emp_no = m.emp_no
LEFT JOIN
    dept_manager dp ON dm.dept_no = dp.dept_no AND dm.to_date = dp.from_date
LEFT JOIN
    employees p ON dp.emp_no = p.emp_no
WHERE
    dm.to_date = '9999-01-01'

;

    ''', con=db_conect)
deparment_managers

# Ejercicio 3

    Obtén la cantidad de empleados que tiene cada manager.

In [ ]:
employees_per_manager = pd.read_sql('''
   SELECT
    CONCAT(m.first_name, ' ', m.last_name) AS manager,
    COUNT(e.emp_no) AS employee_count
FROM
    employees e
JOIN
    dept_manager dm ON e.emp_no = dm.emp_no
JOIN
    employees m ON dm.emp_no = m.emp_no
GROUP BY
    manager



    ;

    ''',  con=db_conect)
employees_per_manager

# Ejercicio 4

    Obtén un listado de los empleados con mayor ingreso y con el menor ingreso por
    cada departamento.
    department | employee_max_salary | e_max_salary | employe_min_salary | e_min_salary

In [ ]:
max_and_min_salary_per_department= pd.read_sql('''
  WITH RankedSalaries AS (
    SELECT
        e.emp_no,
        e.first_name,
        e.last_name,
        d.dept_name AS department,
        s.salary,
        ROW_NUMBER() OVER (PARTITION BY d.dept_name ORDER BY s.salary DESC) AS SalaryRankDesc,
        ROW_NUMBER() OVER (PARTITION BY d.dept_name ORDER BY s.salary ASC) AS SalaryRankAsc
    FROM
        employees e
    JOIN
        dept_emp de ON e.emp_no = de.emp_no
    JOIN
        departments d ON de.dept_no = d.dept_no
    JOIN
        salaries s ON e.emp_no = s.emp_no
)
SELECT
    emp_no,
    first_name,
    last_name,
    department,
    salary
FROM
    RankedSalaries
WHERE
    SalaryRankDesc <= 10 OR SalaryRankAsc <= 10;

''', con= db_conect)
max_and_min_salary_per_department

# Ejercicio 5

    Obtén un listado de todos los cargos con la cantidad de empleados y a su vez
    mostrar el empleado más antiguo y el más nuevo.
    titles | employees | former_employee | new_employee

In [ ]:

older_and_new_employee = pd.read_sql(''' 
    WITH RankedEmployees AS (
    SELECT
        t.title AS Title,
        e.emp_no,
        RANK() OVER (PARTITION BY t.title ORDER BY e.hire_date ASC) AS HireRankAsc,
        RANK() OVER (PARTITION BY t.title ORDER BY e.hire_date DESC) AS HireRankDesc
    FROM
        titles t
    JOIN
        employees e ON t.emp_no = e.emp_no
)

SELECT
    re.Title,
    COUNT(DISTINCT re.emp_no) AS Employees,
    MIN(CASE WHEN HireRankAsc = 1 THEN re.emp_no END) AS former_Employee,
    MAX(CASE WHEN HireRankDesc = 1 THEN re.emp_no END) AS new_Employee
FROM
    RankedEmployees re
GROUP BY
    re.Title
ORDER BY
    Employees DESC

    ;
    '''
    , con=db_conect)
older_and_new_employee